<a href="https://colab.research.google.com/github/19521309/19521309/blob/main/vgg16_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# For commands
import os
os.chdir('/content/')
import time
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
# For array manipulation
import numpy as np
import pandas as pd
import pandas.util.testing as tm
# For visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import cv2
import imageio as io
from pylab import *
from sklearn.manifold import TSNE
#For model performance
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
#For model training
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image

BƯỚC 1: Load pretrained model VGG16

In [1]:
from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()
model.summary()

553476096/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

Bước 2: Load các ảnh trong 1 thư mục

In [3]:
data_path = "/content/sample_data/dataset"

In [4]:
train = os.path.join(data_path)

train_images = sorted(os.listdir(train))
print("Total number of images in the training set: ", len(train_images))

Total number of images in the training set:  16


Bước 3: Rút trích đặc trưng các ảnh trong thư mục

In [11]:
filenames = os.listdir('/content/sample_data/dataset')
files = []
categories = []
for file in filenames:
    category = file.split('_')[0]
    files.append('/content/sample_data/dataset' + file)
    categories.append(category)
        
        
df = pd.DataFrame({
    'filename': files,
    'category': categories
})

In [12]:
df.head()

,filename,category
0,/content/sample_data/datasetdog_3.jpg,dog
1,/content/sample_data/datasetcat_2.jpg,cat
2,/content/sample_data/dataset.ipynb_checkpoints,.ipynb
3,/content/sample_data/datasetcar_2.jpg,car
4,/content/sample_data/datasetcat_1.jpg,cat


In [13]:
df['category'].value_counts()

dog       3
human     3
cat       3
tree      3
car       3
.ipynb    1
Name: category, dtype: int64

In [14]:
df.shape

(16, 2)

Bước 4: đọc ảnh truy vấn

In [15]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

In [16]:
from PIL import Image
from keras.preprocessing import image

In [18]:
image = load_img('/content/sample_data/dataset/dog_1.jpg', target_size=(224, 224))


Bước 5: Rút trích đặc trưng ảnh truy vấn

In [19]:
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))


49152/35363 [=========================================] - 0s 0us/step
Samoyed (91.79%)


Bước 6: Dùng mô hình KNN search với top K = 3 để tìm các đặc trưng gần với đặc trưng truy vấn nhất

In [26]:
dataset = np.array(categories)
dataset.shape
dataset = dataset.reshape(-1,1)

In [30]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=3)
knn.fit(categories.reshape(-1, 1))

AttributeError: ignored

Bước 7: show top K ảnh gần nhất đó